# 0.0 Imports

In [1]:
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt
import seaborn           as sns

# 1.0 Helper Functions

# 2.0 Data Collection

Data from Kaggle Airbnb competition: https://www.kaggle.com/competitions/airbnb-recruiting-new-user-bookings/data

In [5]:
df_users_raw = pd.read_csv("../data/train_users_2.csv")
df_countries_raw = pd.read_csv("../data/countries.csv")
df_age_gender_raw = pd.read_csv("../data/age_gender_bkts.csv")
df_sessions_raw = pd.read_csv("../data/sessions.csv", low_memory=True)

# 3.0 Data description

## Users 

#### Data Dictionary

* <b>id</b>: user id
* <b>date_account_created</b>: the date of account creation
* <b>timestamp_first_active</b>: timestamp of the first activity, note that it can be earlier than date_account_created or date_first_booking because a user can search before signing up
* <b>date_first_booking</b>: date of first booking
* <b>gender</b>: user gender
* <b>age</b>: user age
* <b>signup_method</b>: what method user used to create the account (basic login and password, using google or facebook)
* <b>signup_flow</b>: the page a user came to signup up from
* <b>language</b>: international language preference
* <b>affiliate_channel</b>: column tells us what type of paid marketing was used. For example, if Airbnb used search engines to attract new users, it would fall under the SEM (Search Engine Marketing) category. Similarly, if Airbnb sent emails to attract new users, it would fall under the email marketing category.
* <b>affiliate_provider</b>:  tells us which company or platform was used for the paid marketing campaign.
* <b>first_affiliate_tracked</b>: whats the first marketing the user interacted with before the signing up
* <b>signup_app</b>: contains the name of the application or website that the user used to create their Airbnb account, such as "Web", "iOS", "Android", or "Moweb"
* <b>first_device_type</b>: type of device used by the user to sign up for their Airbnb account, such as a desktop computer, smartphone, tablet, or other type of device.
* <b>first_browser</b>:  contains the name of the web browser that the user used to sign up for their Airbnb account, such as Google Chrome, Mozilla Firefox, Safari, Internet Explorer, or others.
* <b>country_destination</b>: this is the target variable you are to predict

In [28]:
df_users_raw.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


## Countries

#### Data Dictionary

* <b>country_destination</b>: country
* <b>lat_destination</b>: latitude of geographical center of the country
* <b>lng_destination</b>: longitude of geographical center of the country
* <b>distance_km</b>: distance between US and the country
* <b>distance_km2</b>: squared distance between US and the country
* <b>destination_language</b>: language of the country
* <b>language_levenshtein_distance</b>: 

In [32]:
df_countries_raw.head()

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,AU,-26.853388,133.275160,15297.7440,7741220.0,eng,0.00
1,CA,62.393303,-96.818146,2828.1333,9984670.0,eng,0.00
2,DE,51.165707,10.452764,7879.5680,357022.0,deu,72.61
3,ES,39.896027,-2.487694,7730.7240,505370.0,spa,92.25
4,FR,46.232193,2.209667,7682.9450,643801.0,fra,92.06


## Population

#### Data Dictionary

In [33]:
df_age_gender_raw.head()

,age_bucket,country_destination,gender,population_in_thousands,year
0,100+,AU,male,1.0,2015.0
1,95-99,AU,male,9.0,2015.0
2,90-94,AU,male,47.0,2015.0
3,85-89,AU,male,118.0,2015.0
4,80-84,AU,male,199.0,2015.0


## Sessions

#### Data Dictionary

In [34]:
df_sessions_raw.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


# 4.0 Data Cleaning

# 5.0 Feature Engineering

# 6.0 EDA

# 7.0 Data Preparation

# 8.0 Feature Selection

# 9.0 Modelling

# 10.0 Hyperparameter Fine Tuning

# 11.0 Final Metrics

# 12.0 Business Performance